In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install boto3

  Using cached s3transfer-0.6.0-py3-none-any.whl (79 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import boto3

def create_collection(collection_id, region):
    client = boto3.client('rekognition', region_name=region)

    # Create a collection
    print('Creating collection:' + collection_id)
    response = client.create_collection(CollectionId=collection_id, 
    Tags={"SampleKey1":"SampleValue1"})
    print('Collection ARN: ' + response['CollectionArn'])
    print('Status code: ' + str(response['StatusCode']))
    print('Done...')

collection_id = 'ecc-rekognition'
region = "us-east-2"
create_collection(collection_id, region)

Creating collection:ecc-rekognition
Collection ARN: aws:rekognition:us-east-2:746366321550:collection/ecc-rekognition
Status code: 200
Done...


In [7]:
import cv2
cam = cv2.VideoCapture(0)
ret, photo = cam.read()
cv2.imwrite("hr.jpg", photo)  
cam.release()

In [8]:
import boto3
import json

def detect_faces(target_file, region):

    client=boto3.client('rekognition', region_name=region)

    imageTarget = open(target_file, 'rb')

    response = client.detect_faces(Image={'Bytes': imageTarget.read()}, 
    Attributes=['ALL'])

    print('Detected faces for ' + photo)
    for faceDetail in response['FaceDetails']:
        print('The detected face is between ' + str(faceDetail['AgeRange']['Low'])
              + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')

        print('Here are the other attributes:')
        print(json.dumps(faceDetail, indent=4, sort_keys=True))

        # Access predictions for individual face details and print them
        print("Gender: " + str(faceDetail['Gender']))
        print("Smile: " + str(faceDetail['Smile']))
        print("Eyeglasses: " + str(faceDetail['Eyeglasses']))
        print("Emotions: " + str(faceDetail['Emotions'][0]))

    return len(response['FaceDetails'])

photo = 'hr.jpg'
region = 'us-east-2'
face_count=detect_faces(photo, region)
print("Faces detected: " + str(face_count))

if face_count == 1:
    print("Image suitable for use in collection.")
else:
    print("Please submit an image with only one face.")

Detected faces for hr.jpg
The detected face is between 16 and 24 years old
Here are the other attributes:
{
    "AgeRange": {
        "High": 24,
        "Low": 16
    },
    "Beard": {
        "Confidence": 99.7911148071289,
        "Value": true
    },
    "BoundingBox": {
        "Height": 0.30886396765708923,
        "Left": 0.40789467096328735,
        "Top": 0.3134382367134094,
        "Width": 0.18920525908470154
    },
    "Confidence": 99.9982681274414,
    "Emotions": [
        {
            "Confidence": 98.9747314453125,
            "Type": "CALM"
        },
        {
            "Confidence": 6.261392116546631,
            "Type": "SURPRISED"
        },
        {
            "Confidence": 5.877046585083008,
            "Type": "FEAR"
        },
        {
            "Confidence": 2.158862829208374,
            "Type": "SAD"
        },
        {
            "Confidence": 0.8766140937805176,
            "Type": "CONFUSED"
        },
        {
            "Confidence": 0.0644

In [13]:
import boto3

def compare_faces(bucket, sourceFile, targetFile, region):
    client = boto3.client('rekognition', region_name=region)

    imageTarget = open(targetFile, 'rb')

    response = client.compare_faces(SimilarityThreshold=99,
                                    SourceImage={'S3Object':{'Bucket':bucket,'Name':sourceFile}},
                                    TargetImage={'Bytes': imageTarget.read()})

    for faceMatch in response['FaceMatches']:
        position = faceMatch['Face']['BoundingBox']
        similarity = str(faceMatch['Similarity'])
        print('The face at ' +
              str(position['Left']) + ' ' +
              str(position['Top']) +
              ' matches with ' + similarity + '% confidence')

    imageTarget.close()
    return len(response['FaceMatches'])

bucket = 'ecc-rekognition'
source_file = '2218-03519.jpg'
target_file = 'hr.jpg'
region = "us-east-2"
face_matches = compare_faces(bucket, source_file, target_file, region)
print("Face matches: " + str(face_matches))

if str(face_matches) == "1":
    print("Face match found.")
else:
    print("No face match found.")

The face at 0.40789467096328735 0.3134382367134094 matches with 99.99359893798828% confidence
Face matches: 1
Face match found.


In [1]:
import boto3

collectionId = 'ecc-rekognition'
region = "us-east-2"
photo = 'hr.jpg'
threshold = 99
maxFaces = 1
client = boto3.client('rekognition', region_name=region)

# input image should be local file here, not s3 file
with open(photo, 'rb') as image:
    response = client.search_faces_by_image(CollectionId=collectionId,
    Image={'Bytes': image.read()},
    FaceMatchThreshold=threshold, MaxFaces=maxFaces)

faceMatches = response['FaceMatches']
print(faceMatches)

for match in faceMatches:
    print('FaceId:' + match['Face']['FaceId'])
    print('ImageId:' + match['Face']['ImageId'])
    print('Similarity: ' + "{:.2f}".format(match['Similarity']) + "%")
    print('Confidence: ' + str(match['Face']['Confidence']))

[{'Similarity': 100.0, 'Face': {'FaceId': '24d3f1f6-5358-487a-8d1f-57d3db463a8e', 'BoundingBox': {'Width': 0.18920500576496124, 'Height': 0.3088639974594116, 'Left': 0.4078949987888336, 'Top': 0.3134379982948303}, 'ImageId': '3c6c960a-76b2-3717-b8ad-48f64e01c6f4', 'ExternalImageId': 'harish', 'Confidence': 99.99829864501953, 'IndexFacesModelVersion': '6.0'}}]
FaceId:24d3f1f6-5358-487a-8d1f-57d3db463a8e
ImageId:3c6c960a-76b2-3717-b8ad-48f64e01c6f4
Similarity: 100.00%
Confidence: 99.99829864501953


In [16]:
import boto3

def add_faces_to_collection(target_file, photo, collection_id, region):
    client = boto3.client('rekognition', region_name=region)

    imageTarget = open(target_file, 'rb')

    response = client.index_faces(CollectionId=collection_id,
                                  Image={'Bytes': imageTarget.read()},
                                  ExternalImageId=photo,
                                  MaxFaces=1,
                                  QualityFilter="AUTO",
                                  DetectionAttributes=['ALL'])
    print(response)

    print('Results for ' + photo)
    print('Faces indexed:')
    for faceRecord in response['FaceRecords']:
        print('  Face ID: ' + faceRecord['Face']['FaceId'])
        print('  Location: {}'.format(faceRecord['Face']['BoundingBox']))
        print('  Image ID: {}'.format(faceRecord['Face']['ImageId']))
        print('  External Image ID: {}'.format(faceRecord['Face']['ExternalImageId']))
        print('  Confidence: {}'.format(faceRecord['Face']['Confidence']))

    print('Faces not indexed:')
    for unindexedFace in response['UnindexedFaces']:
        print(' Location: {}'.format(unindexedFace['FaceDetail']['BoundingBox']))
        print(' Reasons:')
        for reason in unindexedFace['Reasons']:
            print('   ' + reason)
    return len(response['FaceRecords'])

image = 'hr.jpg'
collection_id = 'ecc-rekognition'
photo_name = 'harish'
region = "us-east-2"

indexed_faces_count = add_faces_to_collection(image, photo_name, collection_id, region)
print("Faces indexed count: " + str(indexed_faces_count))

{'FaceRecords': [{'Face': {'FaceId': '24d3f1f6-5358-487a-8d1f-57d3db463a8e', 'BoundingBox': {'Width': 0.18920525908470154, 'Height': 0.30886396765708923, 'Left': 0.40789467096328735, 'Top': 0.3134382367134094}, 'ImageId': '3c6c960a-76b2-3717-b8ad-48f64e01c6f4', 'ExternalImageId': 'harish', 'Confidence': 99.9982681274414}, 'FaceDetail': {'BoundingBox': {'Width': 0.18920525908470154, 'Height': 0.30886396765708923, 'Left': 0.40789467096328735, 'Top': 0.3134382367134094}, 'AgeRange': {'Low': 16, 'High': 24}, 'Smile': {'Value': False, 'Confidence': 95.15637969970703}, 'Eyeglasses': {'Value': False, 'Confidence': 97.49545288085938}, 'Sunglasses': {'Value': False, 'Confidence': 99.99663543701172}, 'Gender': {'Value': 'Male', 'Confidence': 99.9888687133789}, 'Beard': {'Value': True, 'Confidence': 99.7911148071289}, 'Mustache': {'Value': False, 'Confidence': 72.88807678222656}, 'EyesOpen': {'Value': True, 'Confidence': 98.34754943847656}, 'MouthOpen': {'Value': False, 'Confidence': 93.114517211

In [17]:
import boto3
import logging
from botocore.exceptions import ClientError

# store local file in S3 bucket
bucket = "ecc-rekognition"
file_name = "hr.jpg"
key_name = "harish"
region = "us-east-2"
s3 = boto3.client('s3', region_name=region)
# Upload the file
try:
    response = s3.upload_file(file_name, bucket, key_name)
    print("File upload successful!")
except ClientError as e:
    logging.error(e)

File upload successful!


In [20]:
import boto3

# Create DynamoDB database with image URL and face data, face ID

def create_dynamodb_table(table_name, region):
    dynamodb = boto3.client("dynamodb", region_name=region)

    table = dynamodb.create_table(
        TableName=table_name,   
        KeySchema=[{
                'AttributeName': 'FaceID', 'KeyType': 'HASH'  # Partition key  
                },],        
            AttributeDefinitions=[
            {
                'AttributeName': 'FaceID', 'AttributeType': 'S'  }, ],        
                ProvisionedThroughput={
            'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10  }
    )
    print(table)
    return table

region = "us-east-2"
database_name = 'ecc-rekognition'
dynamodb_table = create_dynamodb_table(database_name, region)
print("Table status:", dynamodb_table)

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'FaceID', 'AttributeType': 'S'}], 'TableName': 'ecc-rekognition', 'KeySchema': [{'AttributeName': 'FaceID', 'KeyType': 'HASH'}], 'TableStatus': 'CREATING', 'CreationDateTime': datetime.datetime(2022, 11, 12, 15, 11, 0, 823000, tzinfo=tzlocal()), 'ProvisionedThroughput': {'NumberOfDecreasesToday': 0, 'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10}, 'TableSizeBytes': 0, 'ItemCount': 0, 'TableArn': 'arn:aws:dynamodb:us-east-2:746366321550:table/ecc-rekognition', 'TableId': '99b68cca-26b6-4a09-a058-6086d1193a2e'}, 'ResponseMetadata': {'RequestId': 'NPPSRP0PSNL6JQUI0PD7K32V1NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 12 Nov 2022 20:11:00 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '500', 'connection': 'keep-alive', 'x-amzn-requestid': 'NPPSRP0PSNL6JQUI0PD7K32V1NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '103422100'}, 'RetryAttempts': 0}}
T

In [28]:
import boto3
from pprint import pprint
from decimal import Decimal
import json

# The local file that was stored in S3 bucket
bucket = "ecc-rekognition"
file_name = "hr.jpg"
key_name = "harish"
region = "us-east-2"
# Get URL of file
file_url = "https://s3.amazonaws.com/{}/{}".format(bucket, key_name)
print(file_url)

# upload face-id, face info, and image url
def AddDBEntry(file_name, file_url, face_id, image_id, confidence):
    dynamodb = boto3.resource('dynamodb', region_name=region)
    table = dynamodb.Table('ecc-rekognition')
    response = table.put_item(
       Item={
            'ExternalImageID': file_name,
            'ImageURL': file_url,
            'FaceID': face_id,
            'ImageID': image_id, 
            'Confidence': json.loads(json.dumps(confidence), parse_float=Decimal)
       }
    )
    return response

# Mock values for face ID, image ID, and confidence - replace them with actual values from your collection results
dynamodb_resp = AddDBEntry(file_name, file_url, "24d3f1f6-5358-487a-8d1f-57d3db463a8e",  
    "3c6c960a-76b2-3717-b8ad-48f64e01c6f4", 99.9982681274414)
print("Database entry successful.")
pprint(dynamodb_resp, sort_dicts=False)

https://s3.amazonaws.com/ecc-rekognition/harish
Database entry successful.
{'ResponseMetadata': {'RequestId': 'NH0D3S0OG25C74K33TTVG81F47VV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'HTTPStatusCode': 200,
                      'HTTPHeaders': {'server': 'Server',
                                      'date': 'Sat, 12 Nov 2022 20:29:30 GMT',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'content-length': '2',
                                      'connection': 'keep-alive',
                                      'x-amzn-requestid': 'NH0D3S0OG25C74K33TTVG81F47VV4KQNSO5AEMVJF66Q9ASUAAJG',
                                      'x-amz-crc32': '2745614147'},
                      'RetryAttempts': 0}}


In [61]:
import cv2
cam = cv2.VideoCapture(0)
ret, photo = cam.read()
cv2.imwrite("harish2.jpg", photo)  
cam.release()

In [62]:
import boto3
import json

def detect_faces(target_file, region):

    client=boto3.client('rekognition', region_name=region)

    imageTarget = open(target_file, 'rb')

    response = client.detect_faces(Image={'Bytes': imageTarget.read()}, 
    Attributes=['ALL'])

    print('Detected faces for ' + photo)
    for faceDetail in response['FaceDetails']:
        print('The detected face is between ' + str(faceDetail['AgeRange']['Low'])
              + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')

        print('Here are the other attributes:')
        print(json.dumps(faceDetail, indent=4, sort_keys=True))

        # Access predictions for individual face details and print them
        print("Gender: " + str(faceDetail['Gender']))
        print("Smile: " + str(faceDetail['Smile']))
        print("Eyeglasses: " + str(faceDetail['Eyeglasses']))
        print("Emotions: " + str(faceDetail['Emotions'][0]))

    return len(response['FaceDetails'])

photo = 'harish2.jpg'
region = 'us-east-2'
face_count=detect_faces(photo, region)
print("Faces detected: " + str(face_count))

if face_count == 1:
    print("Image suitable for use in collection.")
else:
    print("Please submit an image with only one face.")

Detected faces for harish2.jpg
The detected face is between 18 and 26 years old
Here are the other attributes:
{
    "AgeRange": {
        "High": 26,
        "Low": 18
    },
    "Beard": {
        "Confidence": 99.95881652832031,
        "Value": true
    },
    "BoundingBox": {
        "Height": 0.35624709725379944,
        "Left": 0.4240448474884033,
        "Top": 0.30750080943107605,
        "Width": 0.21639052033424377
    },
    "Confidence": 99.9974365234375,
    "Emotions": [
        {
            "Confidence": 99.9200668334961,
            "Type": "CALM"
        },
        {
            "Confidence": 6.252177715301514,
            "Type": "SURPRISED"
        },
        {
            "Confidence": 5.874932765960693,
            "Type": "FEAR"
        },
        {
            "Confidence": 2.1503427028656006,
            "Type": "SAD"
        },
        {
            "Confidence": 0.053955305367708206,
            "Type": "CONFUSED"
        },
        {
            "Confidence

In [63]:
import boto3

bucket = 'ecc-rekognition'
collectionId = 'ecc-rekognition'
region = "us-east-2"
fileName = 'harish2.jpg'
threshold = 70
maxFaces = 1
client = boto3.client('rekognition', region_name=region)

# input image should be local file here, not s3 file
with open(fileName, 'rb') as image:
    response = client.search_faces_by_image(CollectionId=collectionId,
    Image={'Bytes': image.read()},
    FaceMatchThreshold=threshold, MaxFaces=maxFaces)

In [64]:
faceMatches = response['FaceMatches']
print(faceMatches)

for match in faceMatches:
    print('FaceId:' + match['Face']['FaceId'])
    print('ImageId:' + match['Face']['ImageId'])
    print('Similarity: ' + "{:.2f}".format(match['Similarity']) + "%")
    print('Confidence: ' + str(match['Face']['Confidence']))

[{'Similarity': 99.99987030029297, 'Face': {'FaceId': '24d3f1f6-5358-487a-8d1f-57d3db463a8e', 'BoundingBox': {'Width': 0.18920500576496124, 'Height': 0.3088639974594116, 'Left': 0.4078949987888336, 'Top': 0.3134379982948303}, 'ImageId': '3c6c960a-76b2-3717-b8ad-48f64e01c6f4', 'ExternalImageId': 'harish', 'Confidence': 99.99829864501953, 'IndexFacesModelVersion': '6.0'}}]
FaceId:24d3f1f6-5358-487a-8d1f-57d3db463a8e
ImageId:3c6c960a-76b2-3717-b8ad-48f64e01c6f4
Similarity: 100.00%
Confidence: 99.99829864501953


## Inappropriate content detection in an image

In [58]:
import cv2
cam = cv2.VideoCapture(0)
ret, photo = cam.read()
cv2.imwrite("hrwhs.jpg", photo)  
cam.release()

In [59]:
import boto3
import logging
from botocore.exceptions import ClientError

# store local file in S3 bucket
bucket = "ecc-rekognition"
file_name = "hrwhs.jpg"
key_name = "hrwhs"
region = "us-east-2"
s3 = boto3.client('s3', region_name=region)
# Upload the file
try:
    response = s3.upload_file(file_name, bucket, key_name)
    print("File upload successful!")
except ClientError as e:
    logging.error(e)

File upload successful!


In [2]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def moderate_image(photo, bucket):

    client=boto3.client('rekognition')

    response = client.detect_moderation_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}})

    print('Detected labels for ' + photo)    
    for label in response['ModerationLabels']:
        print (label['Name'] + ' : ' + str(label['Confidence']))
        print (label['ParentName'])
    return len(response['ModerationLabels'])



def main():
    photo='hrmod'
    bucket='ecc-rekognition'
    label_count=moderate_image(photo, bucket)
    print("Labels detected: " + str(label_count))


if __name__ == "__main__":
    main()
 


Detected labels for hrmod
Middle Finger : 92.08779907226562
Rude Gestures
Rude Gestures : 92.08779907226562

Labels detected: 2


## PPE Detection

In [45]:
import cv2
cam = cv2.VideoCapture(0)
ret, photo = cam.read()
cv2.imwrite("hrmaskhand.jpg", photo)  
cam.release()

In [46]:
import boto3
import logging
from botocore.exceptions import ClientError

# store local file in S3 bucket
bucket = "ecc-rekognition"
file_name = "hrmaskhand.jpg"
key_name = "hrmaskhand"
region = "us-east-2"
s3 = boto3.client('s3', region_name=region)
# Upload the file
try:
    response = s3.upload_file(file_name, bucket, key_name)
    print("File upload successful!")
except ClientError as e:
    logging.error(e)

File upload successful!


In [47]:
#Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def detect_labels(photo, bucket):

    client=boto3.client('rekognition')

    response = client.detect_protective_equipment(Image={'S3Object':{'Bucket':bucket,'Name':photo}}, 
        SummarizationAttributes={'MinConfidence':80, 'RequiredEquipmentTypes':['FACE_COVER', 'HAND_COVER', 'HEAD_COVER']})
        
 
    print('Detected PPE for people in image ' + photo) 
    print('\nDetected people\n---------------')   
    for person in response['Persons']:
        
        print('Person ID: ' + str(person['Id']))
        print ('Body Parts\n----------')
        body_parts = person['BodyParts']
        if len(body_parts) == 0:
                print ('No body parts found')
        else:
            for body_part in body_parts:
                print('\t'+ body_part['Name'] + '\n\t\tConfidence: ' + str(body_part['Confidence']))
                print('\n\t\tDetected PPE\n\t\t------------')
                ppe_items = body_part['EquipmentDetections']
                if len(ppe_items) ==0:
                    print ('\t\tNo PPE detected on ' + body_part['Name'])
                else:    
                    for ppe_item in ppe_items:
                        print('\t\t' + ppe_item['Type'] + '\n\t\t\tConfidence: ' + str(ppe_item['Confidence'])) 
                        print('\t\tCovers body part: ' + str(ppe_item['CoversBodyPart']['Value']) + '\n\t\t\tConfidence: ' + str(ppe_item['CoversBodyPart']['Confidence']))
                        print('\t\tBounding Box:')
                        print ('\t\t\tTop: ' + str(ppe_item['BoundingBox']['Top']))
                        print ('\t\t\tLeft: ' + str(ppe_item['BoundingBox']['Left']))
                        print ('\t\t\tWidth: ' +  str(ppe_item['BoundingBox']['Width']))
                        print ('\t\t\tHeight: ' +  str(ppe_item['BoundingBox']['Height']))
                        print ('\t\t\tConfidence: ' + str(ppe_item['Confidence']))
            print()
        print()

    print('Person ID Summary\n----------------')
    display_summary('With required equipment',response['Summary']['PersonsWithRequiredEquipment'] )
    display_summary('Without required equipment',response['Summary']['PersonsWithoutRequiredEquipment'] )
    display_summary('Indeterminate',response['Summary']['PersonsIndeterminate'] )
   
    print()
    return len(response['Persons'])

#Display summary information for supplied summary.
def display_summary(summary_type, summary):
    print (summary_type + '\n\tIDs: ',end='')
    if (len(summary)==0):
        print('None')
    else:
        for num, id in enumerate(summary, start=0):
            if num==len(summary)-1:
                print (id)
            else:
                print (str(id) + ', ' , end='')



def main():
    photo='hrmaskhand'
    bucket='ecc-rekognition'
    person_count=detect_labels(photo, bucket)
    print("Persons detected: " + str(person_count))


if __name__ == "__main__":
    main()




Detected PPE for people in image hrmaskhand

Detected people
---------------
Person ID: 0
Body Parts
----------
	FACE
		Confidence: 99.99797821044922

		Detected PPE
		------------
		FACE_COVER
			Confidence: 99.96553802490234
		Covers body part: True
			Confidence: 98.20243835449219
		Bounding Box:
			Top: 0.4151860773563385
			Left: 0.42229634523391724
			Width: 0.1454888880252838
			Height: 0.19186776876449585
			Confidence: 99.96553802490234
	LEFT_HAND
		Confidence: 98.40178680419922

		Detected PPE
		------------
		No PPE detected on LEFT_HAND
	HEAD
		Confidence: 100.0

		Detected PPE
		------------
		No PPE detected on HEAD


Person ID Summary
----------------
With required equipment
	IDs: None
Without required equipment
	IDs: 0
Indeterminate
	IDs: None

Persons detected: 1


## Text Detection

In [49]:
import boto3
import logging
from botocore.exceptions import ClientError

# store local file in S3 bucket
bucket = "ecc-rekognition"
file_name = "download.png"
key_name = "plate"
region = "us-east-2"
s3 = boto3.client('s3', region_name=region)
# Upload the file
try:
    response = s3.upload_file(file_name, bucket, key_name)
    print("File upload successful!")
except ClientError as e:
    logging.error(e)

File upload successful!


In [50]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def detect_text(photo, bucket):

    client=boto3.client('rekognition')

    response=client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':photo}})
                        
    textDetections=response['TextDetections']
    print ('Detected text\n----------')
    for text in textDetections:
            print ('Detected text:' + text['DetectedText'])
            print ('Confidence: ' + "{:.2f}".format(text['Confidence']) + "%")
            print ('Id: {}'.format(text['Id']))
            if 'ParentId' in text:
                print ('Parent Id: {}'.format(text['ParentId']))
            print ('Type:' + text['Type'])
            print()
    return len(textDetections)

def main():

    bucket='ecc-rekognition'
    photo='plate'
    text_count=detect_text(photo,bucket)
    print("Text detected: " + str(text_count))


if __name__ == "__main__":
    main()

Detected text
----------
Detected text:KL 65 H 4383
Confidence: 99.02%
Id: 0
Type:LINE

Detected text:KL
Confidence: 99.84%
Id: 1
Parent Id: 0
Type:WORD

Detected text:65
Confidence: 99.82%
Id: 2
Parent Id: 0
Type:WORD

Detected text:H
Confidence: 96.41%
Id: 3
Parent Id: 0
Type:WORD

Detected text:4383
Confidence: 100.00%
Id: 4
Parent Id: 0
Type:WORD

Text detected: 5
